In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [2]:
from google.colab import files
uploaded = files.upload()


Saving shape_todya.py to shape_todya.py


In [ ]:
uploaded = files.upload()

Saving text_new.py to text_new.py


In [4]:
# Load the file
%run '/content/shape_todya.py'  # Adjust path accordingly

# Assuming you have an image path
image_path = '/content/Test2.jpg'  # Adjust path accordingly

# Call the function
ex_shape, ex_coor = detect_shapes(image_path)

# Print or use the results|
print("Detected Shapes:", ex_shape)
print("Coordinates for Shapes:", ex_coor)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 93.2MB/s]


Detected Shapes: ['arrow_line_down', 'arrow_line_down', 'arrow_line_down', 'start_end', 'start_end', 'arrow_line_down', 'decision', 'arrow_line_right', 'scan', 'arrow_line_down', 'arrow_line_right', 'scan', 'scan', 'arrow_line_left', 'arrow_line_down', 'process', 'arrow_line_down']
Coordinates for Shapes: [{'xmin': 288, 'ymin': 117, 'xmax': 328, 'ymax': 191}, {'xmin': 227, 'ymin': 834, 'xmax': 259, 'ymax': 897}, {'xmin': 233, 'ymin': 288, 'xmax': 274, 'ymax': 370}, {'xmin': 248, 'ymin': 22, 'xmax': 398, 'ymax': 122}, {'xmin': 181, 'ymin': 889, 'xmax': 369, 'ymax': 1016}, {'xmin': 510, 'ymin': 824, 'xmax': 547, 'ymax': 948}, {'xmin': 106, 'ymin': 348, 'xmax': 393, 'ymax': 550}, {'xmin': 364, 'ymin': 926, 'xmax': 556, 'ymax': 958}, {'xmin': 398, 'ymin': 707, 'xmax': 652, 'ymax': 845}, {'xmin': 265, 'ymin': 541, 'xmax': 304, 'ymax': 718}, {'xmin': 380, 'ymin': 425, 'xmax': 525, 'ymax': 446}, {'xmin': 100, 'ymin': 712, 'xmax': 373, 'ymax': 857}, {'xmin': 126, 'ymin': 179, 'xmax': 477, 'yma

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
script_code = """

# Import necessary libraries
import os
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import numpy as np
from google.colab import drive
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

# Mount Google Drive
drive.mount('/content/drive')

# Define the function to extract text and coordinates
def extract_text_and_coordinates(image_path):
    class TextClassifier:
        def __init__(self):
            # Initialize keras-ocr pipeline
            self.pipeline = keras_ocr.pipeline.Pipeline()

        def __set_image(self, image_path):
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            blur = cv2.GaussianBlur(image, (5, 5), 0)
            _, image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            return image

        def __get_bbox(self, image_path):
            images = keras_ocr.tools.read(image_path)
            self.image = images
            prediction_groups = self.pipeline.recognize([images])
            texts = []
            results = []

            for ibox in prediction_groups[0]:
                box = ibox[1]
                texts.append(ibox[0])
                xs, ys = set(), set()
                for x in box:
                    xs.add(x[0])
                    ys.add(x[1])
                results.append(list(map(int, [min(xs), min(ys), max(xs), max(ys)]))) # ymin, xmin, ymax, xmax

            return results, texts

        def recognize(self, image_path):
            boxes, texts = self.__get_bbox(image_path)
            nodes = []

            for box, text in zip(boxes, texts):
                xmin, ymin, xmax, ymax = box
                nodes.append({'text': text, 'coordinate': [xmin, xmax, ymin, ymax]})

            return nodes

    # Initialize the classifier
    classifier = TextClassifier()

    # Recognize text in the image
    nodes = classifier.recognize(image_path)

    # Extract text and coordinates into separate arrays
    ex_text = [node['text'] for node in nodes]
    ex_co = [node['coordinate'] for node in nodes]

    return ex_text, ex_co

# Define the function to correct text
def correct_text(text_array):
    # Initialize SymSpell object
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

    # Load the dictionary
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

    corrected_text_array = []
    for text in text_array:
        suggestions = sym_spell.lookup(text, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_text_array.append(suggestions[0].term)
        else:
            corrected_text_array.append(text)
    return corrected_text_array

# Main function
def main(image_path):
    # Extract text and coordinates
    ex_text, ex_co = extract_text_and_coordinates(image_path)

    # Correct the extracted text
    cr_text = correct_text(ex_text)

    # Print the results
    print("Extracted Texts:", ex_text)
    print("Corrected Texts:", cr_text)
    print("Extracted Coordinates:", ex_co)

    # Optionally, display the image with bounding boxes
    image = cv2.imread(image_path)
    for coord in ex_co:
        xmin, xmax, ymin, ymax = coord
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        cv2.putText(image, ex_text[ex_co.index(coord)], (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    return ex_text, cr_text, ex_co
"""

# Save the script code to a .py file
with open('/content/drive/MyDrive/ocr_script.py', 'w') as f:
    f.write(script_code)


In [6]:
!pip install tensorflow keras-ocr opencv-python numpy symspellpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.4-cp310-cp310-linux_x86_64.whl size=187494 sha256=0537832c94a5e06a36853b29c9794dad30d45fed236ac2ed46ebd6f25974a59d
  Stored in directory: /root/.cache/pip/wheels/4c/0f/10/c20d67cd765ee5b3666d759a307241bba0663135d6ee1c0072
Successfully built editdi

In [41]:
# Import necessary libraries
import os
import matplotlib.pyplot as plt
import keras_ocr
import cv2
import numpy as np
from google.colab import drive
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources

# Mount Google Drive
#drive.mount('/content/drive')

class OCRProcessor:
    def __init__(self):
        # Create a pipeline for OCR processing
        self.pipeline = keras_ocr.pipeline.Pipeline()

    def __get_bbox(self, image_path):
        try:
            # Read the image using OpenCV
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Image at path {image_path} could not be read.")

            # Convert the image to RGB (keras-ocr expects RGB images)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Use the OCR pipeline to detect text
            images = keras_ocr.tools.read(image_path)
            self.image = images
            prediction_groups = self.pipeline.recognize([images])

            if not prediction_groups or not prediction_groups[0]:
                return [], []

            # Extract the bounding boxes and text
            texts = []
            results = []
            for text, box in prediction_groups[0]:
                texts.append(text)
                xs, ys = set(), set()
                for x in box:
                    xs.add(x[0])
                    ys.add(x[1])
                results.append(list(map(int, [min(xs), min(ys), max(xs), max(ys)])))  # ymin, xmin, ymax, xmax

            return texts, results
        except Exception as e:
            print(f"An error occurred in __get_bbox: {e}")
            return [], []

    def process_image(self, image_path):
        return self.__get_bbox(image_path)

# Define the function to correct text
def correct_text(text_array):
    # Initialize SymSpell object
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

    # Load the dictionary
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

    corrected_text_array = []
    for text in text_array:
        suggestions = sym_spell.lookup(text, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_text_array.append(suggestions[0].term)
        else:
            corrected_text_array.append(text)
    return corrected_text_array

# Main function
def main(image_path):
    ocr_processor = OCRProcessor()
    ex_text, ex_co = ocr_processor.process_image(image_path)
    if not ex_text:
        print(f"No text detected in the image at path {image_path}.")
        return [], [], []

    # Correct the extracted text
    cr_text = correct_text(ex_text)

    # Print the results
    print("Extracted Texts:", ex_text)
    print("Corrected Texts:", cr_text)
    print("Extracted Coordinates:", ex_co)

    # Optionally, display the image with bounding boxes
    # image = cv2.imread(image_path)
    # for coord in ex_co:
    #     xmin, xmax, ymin, ymax = coord
    #     cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
    #     cv2.putText(image, ex_text[ex_co.index(coord)], (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # plt.figure(figsize=(10, 10))
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # #plt.show()

    return ex_text, cr_text, ex_co

# Example usage (you can update the image path as needed)
image_path = '/content/4107.jpg'
ex_text, cr_text, ex_co = main(image_path)

ex_shape, ex_coor = detect_shapes(image_path)

# Print or use the results
print("Detected Shapes:", ex_shape)
print("Coordinates for Shapes:", ex_coor)




Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 4s 4s/step
Extracted Texts: ['start', 'input', 'number', 'no', 'ls', 'number', '0', '2e0', 'yes', 'output', 'output', 'even', 'odd', 'end']
Corrected Texts: ['start', 'input', 'number', 'no', 'is', 'number', 'a', 'be', 'yes', 'output', 'output', 'even', 'odd', 'end']
Extracted Coordinates: [[82, 28, 110, 41], [80, 117, 110, 129], [73, 132, 118, 143], [234, 237, 251, 249], [89, 243, 101, 253], [52, 256, 94, 267], [96, 257, 108, 268], [109, 257, 142, 268], [101, 319, 122, 331], [78, 368, 113, 379], [254, 368, 289, 379], [81, 382, 109, 394], [260, 382, 285, 395], [84, 460, 106, 471]]
Detected Shapes: ['start_end', 'start_end', 'arrow_line_down', 'arrow_line_down', 'arrow_line_down', 'arrow_line_down', 'decision', 'process', 'arrow_line_left', 'scan', 'scan', 'arrow_line_down', 'process', 'scan', 'arrow_line_right']
Coordinates for Shapes: [{'xmin': 39, 'ymin': 

In [42]:
def map_text_to_shapes(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates):
    # Initialize a dictionary to store the texts for each shape
    shape_texts = {i: [] for i in range(len(detected_shapes))}

    # Helper function to check if a coordinate is inside or near a shape's bounding box
    def is_inside_or_near(coord, shape_coord, threshold=10):
        x, y, x2, y2 = coord[0], coord[1], coord[2], coord[3]
        xmin, ymin, xmax, ymax = shape_coord['xmin'], shape_coord['ymin'], shape_coord['xmax'], shape_coord['ymax']
        return (xmin - threshold <= x <= xmax + threshold and
                ymin - threshold <= y <= ymax + threshold and
                xmin - threshold <= x2 <= xmax + threshold and
                ymin - threshold <= y2 <= ymax + threshold)

    # Helper function to check if one shape is inside or overlaps significantly with another
    def is_significantly_overlapping(shape_coord1, shape_coord2):
        xmin1, ymin1, xmax1, ymax1 = shape_coord1['xmin'], shape_coord1['ymin'], shape_coord1['xmax'], shape_coord1['ymax']
        xmin2, ymin2, xmax2, ymax2 = shape_coord2['xmin'], shape_coord2['ymin'], shape_coord2['xmax'], shape_coord2['ymax']

        # Check if shape_coord1 is completely inside shape_coord2
        if (xmin2 <= xmin1 <= xmax2 and ymin2 <= ymin1 <= ymax2 and
            xmin2 <= xmax1 <= xmax2 and ymin2 <= ymax1 <= ymax2):
            return True

        # Check if shape_coord2 is completely inside shape_coord1
        if (xmin1 <= xmin2 <= xmax1 and ymin1 <= ymin2 <= ymax1 and
            xmin1 <= xmax2 <= xmax1 and ymin1 <= ymax2 <= ymax1):
            return True

        # Check for significant overlap
        overlap_x = max(0, min(xmax1, xmax2) - max(xmin1, xmin2))
        overlap_y = max(0, min(ymax1, ymax2) - max(ymin1, ymin2))
        area_overlap = overlap_x * overlap_y
        area_shape1 = (xmax1 - xmin1) * (ymax1 - ymin1)
        area_shape2 = (xmax2 - xmin2) * (ymax2 - ymin2)

        # Consider significant if overlap is more than 50% of either shape
        return area_overlap / area_shape1 > 0.5 or area_overlap / area_shape2 > 0.5

    # Iterate through each text and its coordinates
    for text, coord in zip(corrected_texts, extracted_coordinates):
        for i, (shape, shape_coord) in enumerate(zip(detected_shapes, shape_coordinates)):
            if shape.startswith('arrow_line'):
                continue  # Ignore arrow lines
            if is_inside_or_near(coord, shape_coord):
                shape_texts[i].append(text)
                break

    # Prepare the final output as a multidimensional array
    final_texts_by_shapes = []
    for i, shape in enumerate(detected_shapes):
        if not shape.startswith('arrow_line'):
            final_texts_by_shapes.append([shape, shape_coordinates[i], shape_texts[i]])

    # Remove redundant shapes (same or overlapping shapes) but keep one
    unique_shapes = []
    redundant_indices = set()
    for i, (shape_name1, shape_coord1, texts1) in enumerate(final_texts_by_shapes):
        if i in redundant_indices:
            continue
        for j, (shape_name2, shape_coord2, texts2) in enumerate(final_texts_by_shapes):
            if i != j and (shape_name1 == 'scan' or shape_name1 == 'process') and (shape_name2 == 'scan' or shape_name2 == 'process'):
                if is_significantly_overlapping(shape_coord1, shape_coord2):
                    if shape_name1 == 'scan':
                        redundant_indices.add(j)  # Prefer scan, mark process as redundant
                    else:
                        redundant_indices.add(i)  # Prefer process, mark scan as redundant

        if i not in redundant_indices:
            unique_shapes.append([shape_name1, shape_coord1, texts1])

    # Sort unique shapes by their ymin value
    unique_shapes.sort(key=lambda x: x[1]['ymin'])

    # Rename shapes based on their position and type
    process_counter = 1
    for i, (shape_name, shape_coord, texts) in enumerate(unique_shapes):
        if shape_name == 'start_end':
            if i == 0:
                unique_shapes[i][0] = 'Start'
            elif i == len(unique_shapes) - 1:
                unique_shapes[i][0] = 'End'
        elif shape_name == 'scan':
            unique_shapes[i][0] = f'Process {process_counter}'
            process_counter += 1
        elif shape_name == 'process':
            unique_shapes[i][0] = f'Process {process_counter}'
            process_counter += 1

    return unique_shapes


In [43]:
def print_mapped_texts(mapped_texts):
    for shape, coordinates, texts in mapped_texts:
        print(f"Shape: {shape}")
        print(f"Coordinates: xmin={coordinates['xmin']}, ymin={coordinates['ymin']}, xmax={coordinates['xmax']}, ymax={coordinates['ymax']}")
        if texts:
            print("Texts:")
            for text in texts:
                print(f" - {text}")
        else:
            print("Texts: None")
        print('-' * 40)

# Get the mapped texts

corrected_texts = cr_text
extracted_coordinates = ex_co
detected_shapes = ex_shape
shape_coordinates = ex_coor
mapped_texts = map_text_to_shapes(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates)

# Print the mapped texts
print_mapped_texts(mapped_texts)

Shape: Start
Coordinates: xmin=39, ymin=6, xmax=150, ymax=63
Texts:
 - start
----------------------------------------
Shape: Process 1
Coordinates: xmin=0, ymin=96, xmax=188, ymax=169
Texts: None
----------------------------------------
Shape: decision
Coordinates: xmin=23, ymin=193, xmax=166, ymax=316
Texts:
 - is
 - number
 - a
 - be
----------------------------------------
Shape: Process 2
Coordinates: xmin=202, ymin=345, xmax=338, ymax=419
Texts:
 - output
 - odd
----------------------------------------
Shape: Process 3
Coordinates: xmin=17, ymin=346, xmax=172, ymax=419
Texts:
 - output
 - even
----------------------------------------
Shape: End
Coordinates: xmin=39, ymin=437, xmax=147, ymax=493
Texts:
 - end
----------------------------------------


In [28]:
def get_connected_arrows_and_destinations(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates):
    def find_final_destination(arrow_coords, visited_arrows):
        """
        Recursively find the final destination shape for the given arrow coordinates.
        """
        arrow_coords_tuple = tuple(arrow_coords.items())
        if arrow_coords_tuple in visited_arrows:
            # Avoid revisiting the same arrow
            return None, None

        visited_arrows.add(arrow_coords_tuple)

        for k, shape in enumerate(detected_shapes):
            if 'arrow_line' not in shape:
                shape_coords = shape_coordinates[k]
                shape_xmin = shape_coords['xmin']
                shape_ymin = shape_coords['ymin']
                shape_xmax = shape_coords['xmax']
                shape_ymax = shape_coords['ymax']

                # Check if the arrow end coordinates are within the bounds of this shape
                if (arrow_coords['xmax'] >= shape_xmin and arrow_coords['xmax'] <= shape_xmax and
                    arrow_coords['ymax'] >= shape_ymin and arrow_coords['ymax'] <= shape_ymax):
                    return shape, shape_coords

        for k, shape in enumerate(detected_shapes):
            if 'arrow_line' in shape:
                arrow_coords_next = shape_coordinates[k]

                # Check if the arrow end coordinates are within the bounds of another arrow
                if (arrow_coords['xmax'] >= arrow_coords_next['xmin'] and
                    arrow_coords['xmax'] <= arrow_coords_next['xmax'] and
                    arrow_coords['ymax'] >= arrow_coords_next['ymin'] and
                    arrow_coords['ymax'] <= arrow_coords_next['ymax']):
                    return find_final_destination(arrow_coords_next, visited_arrows)
        return None, None

    def label_arrow(arrow_coords):
        """
        Identify if an arrow is labeled with 'yes' or 'no'.
        """
        for text, coord in zip(corrected_texts, extracted_coordinates):
            if text.lower() in ['yes', 'no']:
                text_xmin, text_ymin, text_xmax, text_ymax = coord
                if (arrow_coords['xmin'] <= text_xmax and arrow_coords['xmax'] >= text_xmin and
                    arrow_coords['ymin'] <= text_ymax and arrow_coords['ymax'] >= text_ymin):
                    return text.lower()
        return None

    def find_nearest_processes_below_or_same_level(dec_coords):
        """
        Find the nearest process shapes below or at the same level as the decision block coordinates.
        """
        dec_xmin, dec_ymin, dec_xmax, dec_ymax = dec_coords['xmin'], dec_coords['ymin'], dec_coords['xmax'], dec_coords['ymax']
        nearest_processes = []

        for i, shape in enumerate(detected_shapes):
            if shape.startswith('Process'):
                shape_coords = shape_coordinates[i]
                shape_ymin = shape_coords['ymin']

                # Consider shapes below or at the same level as the decision block
                if shape_ymin >= dec_ymin:
                    nearest_processes.append((shape, shape_coords))

        return sorted(nearest_processes, key=lambda x: x[1]['ymin'])

    # Initialize a dictionary to store arrows connected to each decision block and their destinations
    decisions_and_arrows_destinations = {}

    # Find all decision blocks
    decision_indices = [i for i, shape in enumerate(detected_shapes) if shape == 'decision']

    if not decision_indices:
        raise ValueError("No decision blocks found in detected shapes.")

    # Iterate through all decision blocks
    for decision_index in decision_indices:
        decision_coordinates = shape_coordinates[decision_index]

        # Coordinates of the decision block
        dec_xmin = decision_coordinates['xmin']
        dec_ymin = decision_coordinates['ymin']
        dec_xmax = decision_coordinates['xmax']
        dec_ymax = decision_coordinates['ymax']

        # Initialize a list to store coordinates of connected arrows and their destinations for the current decision block
        connected_arrows_destinations = []

        # Check each shape for connected arrows
        for i, shape in enumerate(detected_shapes):
            if 'arrow_line' in shape:
                # Get coordinates of the current arrow
                arrow_coords = shape_coordinates[i]
                arrow_xmin = arrow_coords['xmin']
                arrow_ymin = arrow_coords['ymin']
                arrow_xmax = arrow_coords['xmax']
                arrow_ymax = arrow_coords['ymax']

                # Check if the arrow is connected to the decision block
                if (arrow_ymin <= dec_ymax and arrow_ymax >= dec_ymin and
                    (arrow_xmax >= dec_xmin and arrow_xmin <= dec_xmax)) or \
                   (arrow_xmin <= dec_xmax and arrow_xmax >= dec_xmin and
                    (arrow_ymax >= dec_ymin and arrow_ymin <= dec_ymax)):

                    # Recursively find the final destination shape for this arrow
                    destination_shape, destination_coordinates = find_final_destination(arrow_coords, set())

                    # Determine if the arrow is labeled 'yes' or 'no'
                    label = label_arrow(arrow_coords)

                    connected_arrows_destinations.append((arrow_coords, destination_shape, destination_coordinates, label))

        # Identify and remove the arrow connected from the top
        top_arrow = None
        for arrow, _, _, _ in connected_arrows_destinations:
            if abs(dec_ymin - arrow['ymax']) <= 15:  # Adjust the tolerance as needed
                top_arrow = arrow
                break

        # Remove the top-connected arrow if found
        if top_arrow:
            connected_arrows_destinations = [ad for ad in connected_arrows_destinations if ad[0] != top_arrow]

        # If fewer than two connected arrows are found, find nearest process shapes below or at the same level
        if len(connected_arrows_destinations) < 2:
            nearest_processes = find_nearest_processes_below_or_same_level(decision_coordinates)
            additional_processes_needed = 2 - len(connected_arrows_destinations)

            # Add the nearest processes below or at the same level until we have two connections
            for i, (shape, coords) in enumerate(nearest_processes[:additional_processes_needed]):
                label = 'yes' if i == 0 else 'no'  # Assign 'yes' to the nearest, 'no' to others
                connected_arrows_destinations.append((None, shape, coords, label))

        # Store the connected arrows and their destinations for the current decision block
        decisions_and_arrows_destinations[tuple(decision_coordinates.values())] = connected_arrows_destinations

    # Return the dictionary containing coordinates of decision blocks and their connected arrows and destinations
    return decisions_and_arrows_destinations


In [ ]:
# def print_connections(connections_dict):
#     for decision_coords, connected_arrows_destinations in connections_dict.items():
#         print(f"Decision block coordinates: {decision_coords}")

#         for arrow_coords, destination_shape, destination_coordinates, label in connected_arrows_destinations:
#             print(f"  Arrow coordinates: {arrow_coords}")
#             print(f"  Destination shape: {destination_shape}")
#             print(f"  Destination coordinates: {destination_coordinates}")
#             print(f"  Label: {label}")
#             print()

#         print("-----------------------")
# connections_dict = get_connected_arrows_and_destinations(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates)

# # Print the connections
print_connections(connections_dict)

In [30]:
def format_output(decision_arrows_destinations):
    formatted_output = []

    for decision, arrows_dest in decision_arrows_destinations.items():
        decision_entry = {}
        decision_entry['Decision'] = decision

        # Initialize lists for if_correct and if_wrong shapes
        if_correct_shapes = []
        if_wrong_shapes = []

        for arrow_coords, destination_shape, destination_coords, label in arrows_dest:
            if label == 'yes':
                if_correct_shapes.append(destination_coords)
            elif label == 'no':
                if_wrong_shapes.append(destination_coords)

        # Handle cases where only one of if_correct or if_wrong is found
        if if_correct_shapes and not if_wrong_shapes:
            # Find the nearest shape to assign as if_wrong
            nearest_shape = find_nearest_shape(decision, if_correct_shapes[0], decision_arrows_destinations)
            if_wrong_shapes.append(nearest_shape)
        elif if_wrong_shapes and not if_correct_shapes:
            # Find the nearest shape to assign as if_correct
            nearest_shape = find_nearest_shape(decision, if_wrong_shapes[0], decision_arrows_destinations)
            if_correct_shapes.append(nearest_shape)
        elif not if_correct_shapes and not if_wrong_shapes:
            # Find a process shape below the decision to assign as if_correct and a nearby shape as if_wrong
            if_correct_shape = find_below_shape(decision, decision_arrows_destinations)
            if_wrong_shape = find_nearest_shape(decision, if_correct_shape, decision_arrows_destinations, exclude=[if_correct_shape])
            if_correct_shapes.append(if_correct_shape)
            if_wrong_shapes.append(if_wrong_shape)

        # Add if_correct and if_wrong entries if they exist
        if if_correct_shapes:
            decision_entry['if_correct'] = if_correct_shapes
        if if_wrong_shapes:
            decision_entry['if_wrong'] = if_wrong_shapes

        formatted_output.append(decision_entry)

    return formatted_output

def find_nearest_shape(decision, reference_shape, decision_arrows_destinations, exclude=[]):
    """
    Find the nearest shape to the reference_shape coordinates that is not in the exclude list.
    """
    nearest_shape = None
    min_distance = float('inf')

    ref_xmin = reference_shape['xmin']
    ref_ymin = reference_shape['ymin']
    ref_xmax = reference_shape['xmax']
    ref_ymax = reference_shape['ymax']

    for _, arrows_dest in decision_arrows_destinations.items():
        for _, destination_shape, destination_coords, _ in arrows_dest:
            if destination_coords not in exclude:
                dest_xmin = destination_coords['xmin']
                dest_ymin = destination_coords['ymin']
                dest_xmax = destination_coords['xmax']
                dest_ymax = destination_coords['ymax']

                # Calculate the Euclidean distance
                distance = ((ref_xmin - dest_xmin) ** 2 + (ref_ymin - dest_ymin) ** 2) ** 0.5
                if distance < min_distance:
                    min_distance = distance
                    nearest_shape = destination_coords

    return nearest_shape

def find_below_shape(decision, decision_arrows_destinations):
    """
    Find a shape below the decision block coordinates.
    """
    dec_xmin = decision['xmin']
    dec_ymin = decision['ymin']
    dec_xmax = decision['xmax']
    dec_ymax = decision['ymax']

    nearest_below_shape = None
    min_vertical_distance = float('inf')

    for _, arrows_dest in decision_arrows_destinations.items():
        for _, destination_shape, destination_coords, _ in arrows_dest:
            dest_xmin = destination_coords['xmin']
            dest_ymin = destination_coords['ymin']
            dest_xmax = destination_coords['xmax']
            dest_ymax = destination_coords['ymax']

            # Check if the shape is below the decision block
            if dest_ymin > dec_ymax:
                vertical_distance = dest_ymin - dec_ymax
                if vertical_distance < min_vertical_distance:
                    min_vertical_distance = vertical_distance
                    nearest_below_shape = destination_coords

    return nearest_below_shape


In [31]:
def generate_pseudocode(flowchart):
    """
    Generate pseudocode from the flowchart shapes.
    Expects a list of lists with each sublist containing [shape_name, coordinates, texts].
    """
    pseudocode = []

    # Extract the start shape
    start_shape = next((shape for shape in flowchart if shape[0] == 'Start'), None)
    if start_shape:
        start_text = ' '.join(start_shape[2])
        pseudocode.append(f"Start: {start_text}")

    # Extract process shapes
    process_shapes = [shape for shape in flowchart if shape[0].startswith('Process')]

    # Check if there's a decision block
    decision_shape = next((shape for shape in flowchart if shape[0] == 'decision'), None)

    if decision_shape:
        # Specific method for decision-based flowchart
        # Extract the first process shape
        input_shape = next((shape for shape in process_shapes if 'input' in shape[2]), None)
        if input_shape:
            input_texts = ' '.join(input_shape[2])
            pseudocode.append(f"Input: {input_texts}")

        # Extract the decision shape
        decision_texts = ' '.join(decision_shape[2])
        pseudocode.append(f"Decision: {decision_texts}")

        # Extract processes following the decision
        process_correct = next((shape for shape in flowchart if shape[0] == 'Process if_correct'), None)
        if process_correct:
            process_correct_texts = ' '.join(process_correct[2])
            pseudocode.append(f"       if correct:\n            {process_correct_texts}")

        process_wrong = next((shape for shape in flowchart if shape[0] == 'Process if_wrong'), None)
        if process_wrong:
            process_wrong_texts = ' '.join(process_wrong[2])
            pseudocode.append(f"       else:\n            {process_wrong_texts}")

    else:
        # No decision block logic
        if len(process_shapes) == 3:
            # Exactly three process shapes
            input_shape, process_shape, output_shape = process_shapes
            input_texts = ' '.join(input_shape[2])
            process_texts = ' '.join(process_shape[2])
            output_texts = ' '.join(output_shape[2])
            pseudocode.append(f"Input: {input_texts}")
            pseudocode.append(f"Process: {process_texts}")
            pseudocode.append(f"Output: {output_texts}")
        elif len(process_shapes) > 3:
            # More than three process shapes
            for i, shape in enumerate(process_shapes, start=1):
                step_texts = ' '.join(shape[2])
                pseudocode.append(f"Step {i}: {step_texts}")

    # Extract the end shape
    end_shape = next((shape for shape in flowchart if shape[0] == 'End'), None)
    if end_shape:
        end_text = ' '.join(end_shape[2])
        pseudocode.append(f"End: {end_text}")

    return '\n'.join(pseudocode)




In [44]:
def coords_to_tuple(coords):
    if coords is None:
        print("Warning: Attempted to convert None to tuple.")
        return None
    return (coords['xmin'], coords['ymin'], coords['xmax'], coords['ymax'])

def process_flowchart_and_generate_pseudocode(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates):
    """
    Main function to process the flowchart and generate pseudocode.
    It manages the flowchart shape identification, connection, and pseudocode generation.
    """

    # Step 1: Map texts to shapes
    shapes = map_text_to_shapes(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates)

    # Step 2: Check for decision blocks
    decision_indices = [i for i, shape in enumerate(detected_shapes) if shape == 'decision']

    if decision_indices:
        # Step 3: Process connections and update shapes if there are decision blocks
        connections_dict = get_connected_arrows_and_destinations(
            corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates
        )

        formatted_output = format_output(connections_dict)

        if formatted_output:  # Check if formatted_output is valid
            for decision_outcome in formatted_output:
                if_correct = decision_outcome.get("if_correct", [])
                if_wrong = decision_outcome.get("if_wrong", [])

                for shape in shapes:
                    shape_coords = coords_to_tuple(shape[1])

                    if shape_coords is None:
                        continue  # Skip updating if shape_coords is None

                    # Update shape based on if_correct coordinates
                    if if_correct:
                        for correct in if_correct:
                            if shape_coords == coords_to_tuple(correct):
                                shape[0] = "Process if_correct"

                    # Update shape based on if_wrong coordinates
                    if if_wrong:
                        for wrong in if_wrong:
                            if shape_coords == coords_to_tuple(wrong):
                                shape[0] = "Process if_wrong"
        else:
            print("Model can't clearly identify the if-else process inside the flowchart.")
    else:
        # No decision blocks found
        print("--No decision blocks found. Proceeding with regular pseudocode generation.")

    # Output updated shapes (debugging)
    for shape in shapes:
        print(shape)

    # Step 4: Generate pseudocode regardless of the presence of decision blocks
    pseudocode = generate_pseudocode(shapes)
    return pseudocode

# Example input data (replace with actual data)
corrected_texts = cr_text
extracted_coordinates = ex_co
detected_shapes = ex_shape
shape_coordinates = ex_coor

# Generate and print pseudocode
pseudocode = process_flowchart_and_generate_pseudocode(corrected_texts, extracted_coordinates, detected_shapes, shape_coordinates)
print("\n")
print("\033[1mThe Generated Pseudocode from the Given Flow Chart Image is:\033[0m")
print(pseudocode)


['Start', {'xmin': 39, 'ymin': 6, 'xmax': 150, 'ymax': 63}, ['start']]
['Process 1', {'xmin': 0, 'ymin': 96, 'xmax': 188, 'ymax': 169}, []]
['decision', {'xmin': 23, 'ymin': 193, 'xmax': 166, 'ymax': 316}, ['is', 'number', 'a', 'be']]
['Process if_wrong', {'xmin': 202, 'ymin': 345, 'xmax': 338, 'ymax': 419}, ['output', 'odd']]
['Process if_correct', {'xmin': 17, 'ymin': 346, 'xmax': 172, 'ymax': 419}, ['output', 'even']]
['End', {'xmin': 39, 'ymin': 437, 'xmax': 147, 'ymax': 493}, ['end']]


The Generated Pseudocode from the Given Flow Chart Image is:
Start: start
Decision: is number a be
       if correct:
            output even
       else:
            output odd
End: end
